# Interacting with GPIO from MicroBlaze

In [1]:
from pynq.overlays.base import BaseOverlay
import time
import multiprocessing
import os
import socket

base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio_pmodb(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function to turn on/off a selected pin of PMODB
void write_gpio_pmodb(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to reset all pins of PMODB
void reset_gpio_pmodb(){
    for (unsigned int i = 0; i < 8; i++){
        write_gpio_pmodb(i, 0);
    }
}

In [3]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio_pmoda(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function to turn on/off a selected pin of PMODA
void write_gpio_pmoda(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to reset all pins of PMODA
void reset_gpio_pmoda(){
    for (unsigned int i = 0; i < 8; i++){
        write_gpio_pmoda(i, 0);
    }
}

In [10]:
def run_server_1():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('127.0.0.1', 12345)) # TODO: replace with client2 IPv4 address
    sock.listen()
    conn, addr = sock.accept()
    with conn:
        print("before1")
        while True:
            print("inside1")
            time.sleep(0.0001)
            tone_freq = 2
            data = conn.recv(1024)
            if (data.decode() == 'emit'):
                print("emit1")
                write_gpio_pmodb(3,1)
                time.sleep((1 / (2 * tone_freq)))
                write_gpio_pmodb(3,0)
                time.sleep((1 / (2 * tone_freq)))
            elif (data.decode() == 'disconnect'):
                print("disconnect1")
                break
    print("after1")

def run_client_1():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    while True:
        time.sleep(0.0001)
        if base.buttons[0].read() != 0:
            sock.connect(('127.0.0.1', 54321)) # TODO: replace with server2 IPv4 address
            break
    while True:
        time.sleep(0.0001)
        if base.buttons[1].read() != 0:
            sock.sendall(b'emit')
        elif base.buttons[3].read() != 0:
            sock.sendall(b'disconnect')
            break
    sock.close()

def run_server_2():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('127.0.0.1', 54321)) # TODO: replace with client1 IPv4 address
    sock.listen()
    conn, addr = sock.accept()
    with conn:
        print("before2")
        while True:
            print("inside2")
            time.sleep(0.0001)
            tone_freq = 2
            data = conn.recv(1024)
            if (data.decode() == 'emit'):
                print("emit2")
                write_gpio_pmoda(3,1)
                time.sleep((1 / (2 * tone_freq)))
                write_gpio_pmoda(3,0)
                time.sleep((1 / (2 * tone_freq)))
            elif (data.decode() == 'disconnect'):
                print("disconnect2")
                break
    print("after2")
            
def run_client_2():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    while True:
        time.sleep(0.0001)
        if base.buttons[0].read() != 0:
            sock.connect(('127.0.0.1', 12345)) # TODO: replace with server1 IPv4 address
            break
    while True:
        time.sleep(0.0001)
        if base.buttons[2].read() != 0:
            sock.sendall(b'emit')
        elif base.buttons[3].read() != 0:
            sock.sendall(b'disconnect')
            break
    sock.close()

p_server_1 = multiprocessing.Process(target=run_server_1)
p_server_1.start()
os.system("taskset -p -c {} {}".format(0, p_server_1.pid))

p_client_1 = multiprocessing.Process(target=run_client_1)
p_client_1.start()
os.system("taskset -p -c {} {}".format(1, p_client_1.pid))

p_server_2 = multiprocessing.Process(target=run_server_2)
p_server_2.start()
os.system("taskset -p -c {} {}".format(1, p_server_2.pid))
            
p_client_2 = multiprocessing.Process(target=run_client_2)
p_client_2.start()
os.system("taskset -p -c {} {}".format(0, p_client_2.pid))

p_server_1.join();
p_client_1.join();
p_server_2.join();
p_client_2.join();

before2
before1
inside1
inside2
emit2
inside2
inside2
inside2
inside2
emit2
inside2
inside2
inside2
emit2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
emit2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
emit2
inside2
inside2
inside2
inside2
emit2
inside2
inside2
inside2
inside2
emit2
inside2
inside2
inside2
inside2
emit2
inside2
inside2
inside2
inside2
emit2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
inside2
emit2
inside2
inside2
inside2
inside2
emit1
inside1
inside1
inside1
inside1
inside1
inside1
inside1
inside1
inside1
inside1
inside1
emit1
inside1
inside1
inside1
inside1
inside1
inside1
inside1
